In [2]:
import json

In [3]:
metadata = json.load(open('instagram-images.json'))
mturk_results = json.load(open('mturk-results.json'))

In [4]:
prediction_files = ['predictions/AWS-Rekognition.json',
                    'predictions/Clarifai.json',
                    'predictions/Google-Cloud-Vision.json',
                    'predictions/Imagga.json',
                    'predictions/Microsoft.json',
                    'predictions/MobiusLabs-Mobile-High-Threshold.json',
                    'predictions/MobiusLabs-mobile-Mid-Threshold.json',                    
                    'predictions/MobiusLabs-mobile-Low-Threshold.json']

In [11]:
results = dict()
for file_path in prediction_files:
    skip_counter = 0
    predictions = json.load(open(file_path))
    name = file_path.split('/')[-1].replace('.json','')    
    results[name] = {'true_postive':0,
                    'false_positve':0,
                    'total':0,
                    'unique_concepts':set()}
    for photo_id, concepts in predictions.items():
        for concept in concepts:
            concept = concept.lower()
            # if concept != concept.encode('ascii', 'ignore'):
            #     # print(u'skipping unicode concept \'{}\' for image {}'.format(concept, photo_id))
            #     skip_counter += 1
            #     continue
            if concept in mturk_results[photo_id]['correct']:
                results[name]['true_postive'] +=1
                results[name]['total'] +=1
                results[name]['unique_concepts'].add(concept)
            elif concept in mturk_results[photo_id]['incorrect']:
                results[name]['false_positve'] +=1
                results[name]['total'] +=1
                results[name]['unique_concepts'].add(concept)                
            else:
                ValueError('Found concept {} for {} for which don\'t have mturk data'.format(concept, photo_id))
    print(f'{skip_counter} concepts are skiped for image: {file_path}')

0 concepts are skiped for image: predictions/AWS-Rekognition.json
{'AWS-Rekognition': {'true_postive': 10043, 'false_positve': 1565, 'total': 11608, 'unique_concepts': {'interior design', 'labrador retriever', 'winter', 'phone', 'bonfire', 'page', 'jogging', 'tie', 'peony', 'cupboard', 'fire', 'underwater', 'canopy', 'trail', 'herbs', 'soap', 'glove', 'blanket', 'newborn', 'candle', 'flower bouquet', 'railway', 'ramp', 'panther', 'bird', 'mansion', 'noodle', 'moon', 'horse', 'seat belt', 'dating', 'waterfowl', 'analog clock', 'weather', 'smoke', 'blouse', 'jar', 'screen', 'scientist', 'windshield', 'lunch', 'ikebana', 'snow', 'lab', 'rust', 'inflatable', 'truck', 'reef', 'magician', 'overcoat', 'clock tower', 'suspension', 'beer', 'herbal', 'american flag', 'laughing', 'grass', 'port', 'hornet', 'nurse', 'computer keyboard', 'table', 'confectionery', 'asleep', 'path', 'ditch', 'cape', 'festival', 'teddy bear', 'horizon', 'sunset', 'apartment building', 'sushi', 'parking', 'jaguar', 'la

In [15]:
print('Name                   Labels/Image   Precision   Unique Concepts')
for name in sorted(results.keys()):
    pretty_name = name.replace('-', ' ').replace('_', ' ').replace(' threshold', '-thresh')
    numbers = results[name]
    labels_per_image = numbers['total'] / float(len(metadata))
    precision = numbers['true_postive'] / float((numbers['total']))
    unique_labels = len(numbers['unique_concepts'])
    print("{:29}{: 6.1f}{: 12.3f}{: 18d}".format(pretty_name, labels_per_image, precision, unique_labels))

Name                   Labels/Image   Precision   Unique Concepts
AWS Rekognition                14.9       0.865              1041
Clarifai                       20.0       0.841              1466
Google Cloud Vision            12.5       0.933              1488
Imagga                         90.1       0.731              4409
Microsoft                      23.6       0.767               559
MobiusLabs mobile high-thresh  13.4       0.934               733
MobiusLabs mobile low-thresh   36.2       0.875              1519
MobiusLabs mobile mid-thresh   26.3       0.899              1224
